<a href="https://www.undp.org/" ><img src="https://upload.wikimedia.org/wikipedia/commons/9/9f/UNDP_logo.svg" style="float:right; max-width: 40px; display: inline" alt="UNDP"/></a> 

# *CoastPred*: example at *Test-site name*

This software is described more in details in the README. 

It enables the users to extract time-series of shoreline change over the last 30+ years at their site of interest and predict its evolution in the future years.
There are six main steps:
1. Retrieval of the satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution
3. Intersection of the shorelines with cross-shore transects
4. Tidal correction 
5. Time series forecasting
4. Extraction of the predicted shorelines

## Initial settings

Refer to the **Installation** section of the README for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. If that step has been completed correctly, the following packages should be imported without any problem.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from python import extract_shorelines as es
from python import analyze_shoreline as asl
from python import correct_tides as ct
from python import predict as pt
from python import reconstruct_shoreline as rs
from python import estimate_pop as ep
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects
os.chdir("../")

## 1. Retrieval of the images from GEE

Define the region of interest (`polygon`), the date range (`dates`) and the satellite missions (`sat_list`) from which you wish to retrieve the satellite images. The images will be cropped on the Google Earth Engine server and only the region of interest will be downloaded as a .tif file. The files will stored in the directory defined in `filepath`. 

Make sure the area of your ROI is smaller than 100 km2 (if larger split it into smaller ROIs).

The function `SDS_download.check_images_available(inputs)` will print the number of images available for your inputs. The Landsat images are divided in Tier 1 and Tier 2, only Tier 1 images can be used for time-series analysis. 

In [2]:
# region of interest (longitude, latitude) - replace the coordinates with your test-site
polygon = [[[0.9840004042206285,5.868659802766709], 
            [1.002491492775628,5.868997660770736],
            [1.003566497639035,5.912882741342837],
            [0.9835488890625466,5.913424406492211],
            [0.9840004042206285,5.868659802766709]]]

# it's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
polygon = SDS_tools.smallest_rectangle(polygon)
# date range
dates = ['2010-01-01', '2022-01-01']
# satellite missions
sat_list = ['S2','L5','L7','L8','L9']
# name of the site
sitename = 'GHANA'
# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data')
# put all the inputs into a dictionnary
inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath}

# before downloading the images, check how many images are available for your inputs
#SDS_download.check_images_available(inputs);

Check whether it is necessary to redefine the metadata or not, i.e. check **if you have already retrieved the images**

In [3]:
# check if there is a 'references' dir containing predefined metadata
is_ref = os.path.isdir(os.path.join(filepath,sitename,'references'))
    
# redefine if files do not exist yet
redefine = not(os.path.exists(os.path.join(filepath,sitename,'%s_metadata.pkl'%(sitename))))
    
# actually we do not redefine the metadata if there are the references data
if (redefine and is_ref):
    redefine = False

# or the user can choose to crash existing files and redefine metadata
# or even force to continue with existing files (/!\ may lead to errors /!\)
# redefine = False ##### TO REMOVE ###########

print('redefine = ',redefine)

redefine =  False



The function `SDS_download.retrieve_images(inputs)` retrives the satellite images from Google Earth Engine.

By default, only Landsat Tier 1 Top-of-Atmosphere and Sentinel-2 Level-1C products are downloaded. 

In case you need to access Tier 2 images for qualitative analysis, you need to set `inputs['include_T2'] = True` before calling `retrieve_images`.

In [4]:
if redefine:
    # this function retrieves the satellite images from Google Earth Engine 
    #inputs['include_T2'] = True
    metadata = SDS_download.retrieve_images(inputs)
else:
    # if images already retrieved, just load the metadata file by only running the function below
    metadata = SDS_download.get_metadata(inputs)

##### 2. Shoreline extraction

This section maps the position of the shoreline on the satellite images. The user can define the cloud threhold (`cloud_thresh`) and select the spatial reference system in which to output the coordinates of the mapped shorelines (`output_epsg`). See http://spatialreference.org/ to find the EPSG number corresponding to your local coordinate system. Make sure that your are using cartesian coordinates and not spherical coordinates (lat,lon) like WGS84. If unsure, use 3857 which is the web mercator projection (used by Google Maps).

To quality control each shoreline detection and manually validate the mapped shorelines, the user has the option to set the parameter `check_detection` to **True**. 
To adjust the position of each shoreline by modifying the threshold defining the sand/water interface you can set `adjust_detection` to **True**. 
Finally, to save a figure for each mapped shoreline as a .jpg in the folder */jpg_files/detection* set `save_figure` to **True**. 

The other parameters are for advanced users only and are described in the README.

In [5]:
settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output  
    'pan_off': True, 
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) for buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 2300,      # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    # add the inputs defined previously
    'inputs': inputs
}

### [OPTIONAL] Save .jpg of the satellite images 
Saves .jpg files of the preprocessed satellite images (cloud masking + pansharpening/down-sampling) under *./data/sitename/jpeg_files\preprocessed*

In [6]:
# options available only if the metadata and the files have to be redefined
if redefine:
    save_jpg = True    

### [OPTIONAL] Digitize a reference shoreline
Creates a reference shoreline which helps to identify outliers and false detections. The reference shoreline is manually digitised by the user on one of the images. The parameter `max_dist_ref` defines the maximum distance from the reference shoreline (in metres) at which a valid detected shoreline can be. If you think that the default value of 100 m will not capture the full shoreline variability of your site, increase this value to an appropriate distance.

In [7]:
# [optional] create a reference shoreline (helps to identify outliers and false detections)
ref_shoreline = True
if ref_shoreline:
    %matplotlib qt
    settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)
    # set the max distance (in meters) allowed from the reference shoreline for a detected shoreline to be valid
    settings['max_dist_ref'] = 200  

Reference shoreline already exists and was loaded


### Batch shoreline detection
Extracts the 2D shorelines from the images in the spatial reference system specified by the user in `'output_epsg'`. The mapped shorelines are saved into `output.pkl` (under *./data/sitename*) and `output.geojson` (to be used in a GIS software).

If you see that the sand pixels on the images are not being identified, change the parameter `sand_color` from `default` to `dark` or `bright` depending on the color of your beach. 

The coordinates are stored in the output dictionnary together with the exact dates in UTC time, the georeferencing accuracy and the cloud cover. This function also removes duplicates and images with inaccurate georeferencing (threhsold at 10m) and makes a simple plot of the mapped shorelines. 

For use in GIS applications, the mapped shorelines are saved as a GEOJSON layer and shapefiles which can be easily imported into QGIS for example. You can choose to save the shorelines as a collection of lines or points (sometimes the lines are crossing over so better to use points). 

In [8]:
if (redefine):
    output = es.extract_shorelines(metadata, settings, inputs, plot=True)

## 3. Shoreline analysis

In this section we show how to compute time-series of cross-shore distance along user-defined shore-normal transects.

**If you have already mapped the shorelines**, just load the output file (`output.pkl`) by running the section below

In [9]:
if not(redefine):
    filepath = os.path.join(inputs['filepath'], sitename)
    with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
        output = pickle.load(f) 

Now we have to **define cross-shore transects** over which to quantify the shoreline changes. Each transect is defined by two points, its **origin (always landward**) and a second point that defines its orientation (so that the transect is normal to the shorelines)

In [10]:
if (redefine):
    # the user will interactively draw the shore-normal transects along the beach by calling:
    transects = SDS_transects.draw_transects(output, settings)
else:
    # the user will load the transect coordinates (make sure the spatial reference system is the same 
    # as defined previously by the parameter *output_epsg*) from a .geojson file by calling:
    #transects = SDS_transects.draw_transects(output, settings)
    geojson_file = os.path.join(inputs['filepath'],sitename, '%s_transects.geojson'%(sitename))
    transects = SDS_tools.transects_from_geojson(geojson_file)

14 transects have been loaded


`analyze_shoreline` intersects the transects with the 2D shorelines to obtain time-series of cross-shore distance.

The time-series of shoreline change for each transect are saved in a .csv file in the data folder (all dates are in UTC time). 

It also plots the location of the transects, make sure they are in the right location with the origin always landwards!

In [11]:
# defines the along-shore distance over which to consider shoreline points to compute the median intersection (robust to outliers)
settings['along_dist'] = 25 
# compute and plot the time series
cross_distance = asl.analyze_shoreline(output,transects,settings,plot=True)

Time-series of the shoreline change along the transects saved as:
E:\UNDP\Coastline prediction tool\to_GitHub\coastsat_dev\data\GHANA\transect_time_series.csv


## 4. Tidal correction

This section shows how to tidally-correct the time-series of shoreline change using time-series of tide level and an estimate of the beach slope.

When using your own file make sure that the dates are in UTC time, as the shorelines are also in UTC, and the datum for the water levels is approx. Mean Sea Level.

We assume that the beach slope at West Point is 0.1 along all transects.

**Note**: if you don't have measured water levels and beach slope, it is possible to obtain an estimate of the beach slope and time-series of modelled tide levels at the time of image acquisition from the [FES2014](https://www.aviso.altimetry.fr/es/data/products/auxiliary-products/global-tide-fes/description-fes2014.html) global tide model by using the [CoastSat.slope](https://github.com/kvos/CoastSat.slope) repository (see [this publication](https://doi.org/10.1029/2020GL088365) for more details, open acess preprint [here](https://www.essoar.org/doi/10.1002/essoar.10502903.1)). Instructions on how to install the global tide model are available [here](https://github.com/kvos/CoastSat.slope/blob/master/doc/FES2014_installation.md).

Apply tidal correction using a linear slope and a reference elevation to which project all the time-series of cross-shore change (to get time-series at Mean Sea Level, set `reference elevation` to 0. You also need an estimate of the beach slope. 

In [12]:
reference_elevation = 0.0 # elevation at which you would like the shoreline time-series to be => MLLW datum = 0 here
beach_slope = 0.5 # replace the slope with value computed in slope module

cross_distance = ct.correct_tides(cross_distance,settings,output,reference_elevation,beach_slope,estimate_slope=False,plot=False)

2013-10-06 10:11:24+00:00
Extracting closest points: 1%2013-11-23 10:11:06+00:00
Extracting closest points: 2%2014-04-16 10:09:27+00:00
Extracting closest points: 4%2014-05-02 10:09:13+00:00
Extracting closest points: 5%2014-05-18 10:09:02+00:00
Extracting closest points: 6%2015-09-10 10:09:24+00:00
Extracting closest points: 8%2015-09-22 10:27:10+00:00
Extracting closest points: 9%2015-09-26 10:09:31+00:00
Extracting closest points: 10%2015-12-11 10:19:05+00:00
Extracting closest points: 12%2015-12-11 10:29:44+00:00
Extracting closest points: 13%2015-12-15 10:09:38+00:00
Extracting closest points: 15%2015-12-21 10:30:09+00:00
Extracting closest points: 16%2016-01-20 10:30:06+00:00
Extracting closest points: 17%2016-03-04 10:09:24+00:00
Extracting closest points: 19%2016-05-09 10:28:43+00:00
Extracting closest points: 20%2016-10-16 10:21:48+00:00
Extracting closest points: 21%2016-10-26 10:25:08+00:00
Extracting closest points: 23%2017-11-10 10:27:51+00:00
Extracting closest points: 24

Rebuild these corrected distances as shorelines and save them as shapefiles.

In [13]:
corrected_sl = rs.reconstruct_shoreline(cross_distance,transects,output['dates'],output,inputs,settings,len(output['dates']),save_corrections=True)

## 5. Computation of the generalization error

The function `cross_validation` computes the **generalization error of the prediction** by splitting the images in train and test samples and comparing the predicted shorelines with shorelines hand-drawn on 5m-resolution images.

In [14]:
# define the number of years to be predicted
n_years_further = 3

message, validity = pt.validate_year(n_years_further, output)


Valid number of years.


In [15]:
# model could be: Holt, ARIMA or SARIMAX
model = 'Holt'

compute_error = True
if compute_error:
    best_param, rmse, mae = pt.model_evaluation(cross_distance, n_years_further, metadata, output, settings, validity, model=model, smooth_data=False, smooth_coef=30, best_param=True, seasonality=2, MNDWI=False)
    print("Best parameters: ",best_param)
    print('======================================================')
    print("Mean absolute error: \n", mae)

C:\Users\Ivana\miniconda3\envs\coastsat_sdg_ai_test2\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Ivana\miniconda3\envs\coastsat_sdg_ai_test2\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Ivana\miniconda3\envs\coastsat_sdg_ai_test2\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Ivana\miniconda3\envs\coastsat_sdg_ai_test2\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Ivana\miniconda3\envs\coastsat_sdg_ai_test2\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warni

Generalization error (mean mae) for the year  1  in the future:  4.770303575327556 

Generalization error (mean mae) for the year  2  in the future:  25.41914184238775 

Generalization error (mean mae) for the year  3  in the future:  107.27553519385322 

Best parameters:  1.0
Mean absolute error: 
 {'1 average per year': {'year 2018': 11.69208648329934, 'year 2019': 63.4748192035013, 'year 2020': 221.5894291109285}, '1': 101.48863887034062, '2 average per year': {'year 2018': 2.6955549551496323, 'year 2019': 39.20289934800899, 'year 2020': 167.36774151823604}, '2': 71.73420503534034, '3 average per year': {'year 2018': 3.1535343169713608, 'year 2019': 36.637536170442786, 'year 2020': 147.688161030357}, '3': 64.24296588696782, '4 average per year': {'year 2018': 5.203266924501895, 'year 2019': 42.839207767710214, 'year 2020': 170.27520007458116}, '4': 74.76559895142503, '5 average per year': {'year 2018': 5.140872613739338, 'year 2019': 43.577348686217626, 'year 2020': 187.317256926997

## 6. Time series forecasting

The function `predict` forecasts the evolution of the shoreline for the next `n_months_further` months. The default predictive model is a **Holt's Linear Trend** model `'Holt'` but you can choose to use an **AutoRegressive Integrated Moving Average model** `'ARIMA'` or an **Seasonal AutoRegressive Integrated Moving Average with eXogenous factors** `'SARIMAX'`.

In [16]:
time_series_pred, dates_pred_m = pt.predict(cross_distance,output,inputs,settings,n_years_further,validity,model=model,param=None,smooth_data=True,smooth_coef=1,seasonality=2,plot=True)


## 7. Extraction of the new shorelines

Reconstruction of the new shoreline. This new shoreline will be plotted and saved as geojson and shape files.

In [ ]:
predicted_sl = rs.reconstruct_shoreline(time_series_pred,transects,dates_pred_m,output,inputs,settings,n_years_further)